# 新聞分類模型

## 1. 爬取中央社新聞

In [1]:
import re
from bs4 import BeautifulSoup
import requests
import html5lib
import json

In [2]:
url = 'https://www.cna.com.tw/list/aspt.aspx'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html5lib')
n=0
labels=[]
while n < len(soup.select_one('#pnProductNavContents > ul').findAll('li')):
    try :
        lab = soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'}).text
        url =soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'})['href']
    except:(TypeError,IndentationError)
    labels.append((url,lab))
    n += 1
labels = list(set(labels))
labels.remove(('http://cnavideo.cna.com.tw', '影音'))
labels.remove(('/list/aall.aspx', '即時'))
labels.append(('https://www.cna.com.tw/list/aall.aspx', '即時'))
#labels.remove(('https://www.cna.com.tw/keyword/九合一.aspx', '九合一'))
labels.remove(('https://www.cna.com.tw/list/sp.aspx', '專題'))
labels.remove(('https://www.cna.com.tw/list/aall.aspx', '即時'))

labs = []
for item in labels:
    item = [item[0],item[1]]

    item[0] = re.search(r'list\/\w+',item[0]).group()[5:]
    labs.append(item)

### 使用payload()post中央社網站分類底下各標題的href連結，整理成labs_href
```python
labs_href=
[
    [ [href,href,href,.....],類別1 ],
    [ [href,href,href,.....],類別2 ],
    [ [href,href,href,.....],類別3 ],
    ...
]
```

In [3]:
def payload(item):
    payload={
        'id': '1056893641102242',
    'ev': 'Microdata',
    'dl': item[0],
    'rl': 'https://www.cna.com.tw/',
    'if': 'false',
    'ts': '1541087113735',
    'cd[Schema.org]': [],
    #'cd[OpenGraph]': {"og:url":"https://www.cna.com.tw/list/ahel.aspx","og:title":"生活 | 中央社 CNA","og:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","og:image:height":"200","og:image:width":"200","og:image":"https://img5.cna.com.tw/www/images/pic_fb.jpg","og:type":"article","article:author":"https://www.facebook.com/cnanewstaiwan","article:publisher":"https://www.facebook.com/cnanewstaiwan"}
    #'cd[Meta]': {"title":"\n\t生活 | 中央社 CNA\n","meta:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","meta:keywords":"News, 新聞, 即時新聞, 中央社"}
    'cd[DataLayer]': [],
    'cd[JSON-LD]': [],
    'sw': '1280',
    'sh': '5000',#'720',
    'v': '2.8.30',
    'r': 'stable',
    'a': 'tmgoogletagmanager',
    'ec': '1',
    'o': '30',
    'fbp': 'fb.2.1541074934894.620990608',
    'it': '1541087112886',
    'coo': 'false',
    'es': 'automatic',
    }
    return payload

In [4]:
labs_href=[]
for item in labs:
    soups=[]
    n=1
    while n < 13:
        url = 'https://www.cna.com.tw/cna2018api/api/simplelist/categorycode/'+item[0]+'/pageidx/'+str(n)
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        body = soup.findAll('body')[0].getText()
        PageUrl = re.findall(r'\"PageUrl\"\:\S[^\,]+',body)
        urls = [txt[11:-1] for txt in PageUrl]
        soups.extend(urls)
        n += 1
        if len(urls) < 20:
            break
    labs_href.append([soups,item[1]])

### 對labs_href中每一個href進行爬蟲抓出新聞標題與內文，並將標題、內文、類別整理成newslist
```python
newslist=
[
    [ [標題,內文],類別 ]
    [ [標題,內文],類別 ]
    [ [標題,內文],類別 ]
    [ [標題,內文],類別 ]
    ...
]

```

In [5]:
news={}
newslist=[]
for lab in labs_href:
    lab_name = lab[1]
    lab_urls = lab[0]
    n = 0
    for url in lab_urls:
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        txt = ""
        try:
            title = soup.find_all('div',{'class':'centralContent'})[0].find_all('h1')[0].getText()
            for s in soup.find_all('div',{'class':'paragraph'})[0].findAll('p'):
                txt=txt+s.getText()
        except:IndexError
        research = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt)
        if research == None or len(research.group())<2:
            print(txt)
        else:
            txt = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt).group()[1:]

        page = [[title,txt],lab_name]
        newslist.append(page)
        news[(title,txt)]=lab_name
        n += 1
        if n % 20 == 0:
            print((lab_name,n),end=",")

('政治', 20),('政治', 40),('政治', 60),('政治', 80),('政治', 100),('證券', 20),('證券', 40),('證券', 60),('證券', 80),('證券', 100),('生活', 20),('生活', 40),('生活', 60),('生活', 80),('生活', 100),('文化', 20),('文化', 40),('文化', 60),('文化', 80),('文化', 100),('運動', 20),('運動', 40),('運動', 60),('運動', 80),('運動', 100),('兩岸', 20),('兩岸', 40),('兩岸', 60),('兩岸', 80),中央社駐香港特派員張謙/12月3日4年前「占中運動」爆發時，香港泛民主派一度氣勢如虹，並於2016年底的立法會選舉取得不錯成績，但近年來形勢逆轉，泛民面對不少內憂壓力。挾著年輕力量的崛起，泛民於2016年9月的立法會選舉大有斬獲，共取得26席。當中包括一人一票的分區直選議席，以及以業界區分的功能組別議席。功能組別以業界人士或代表區分，從來都是泛民較弱的版塊，但當年泛民在這方面獲得的選票大增，顯示其勢力正往業界延伸，威脅親政府建制派的基本盤。但「占中」之後，當局在北京中央高喊必須對「港獨」零容忍的情況下，開始以行政手段限制立場較為偏激的泛民人士參選。主要要求各候選人在參選前，必須簽署一份參選聲明，表明自己擁護基本法和保證效忠香港特別行政區。這項新規定導致一些泛民代表、特別是主張「港獨」的年輕代表，無法參加當年的立法會選舉，甚至讓新成立的年輕政黨香港眾志，一夜之間被排除在選舉之外，無法透過參選而進入香港管治架構。選後，當局也對泛民當選者窮追猛打，以他們在宣誓就任時未符合基本法所規定的「莊嚴」要求，向法院提出聲請，要求褫奪他們的資格，結果導致泛民6人喪失議席。進入2018年，泛民的困境並未消除。在立法會議員補選中未能拿回失去的全部議席，僅取回3席，失去另外2席，讓泛民在議會的人數減少。今年3月和11月，當局為泛民失去的其中5個議席進行補選，其中3月補選4人，11月補選1人。結果泛民在3月只贏回兩席，失去另外2席。更重要的是，泛民在一人一票的地方直選，被建制派打敗而失去1席，這是泛民和建制派歷來在一對一的對決下，首次落敗。在上月的補選中，

## 2. 找出所有類別的高頻字

### 將newslist各種分類的標題與內文整合成title_tags , txt_tags
```python
title_tags=
{
    類別 : 全標題文字
}

txt_tags=
{
    類別 : 全內文文字
}
```

In [6]:
import jieba
import nltk

In [7]:
stopwords=[]
with open('../30/txt/stopwords_ch.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopwords.append(data)

In [8]:
lab_tags ={}
prelab = ''
books = list(news.items())
for book in books:
    for lab in ['生活','運動','政治','科技','國際','兩岸','產經','社會','地方','娛樂','證券','文化']:
        if book[1]==lab:
            if prelab != lab:
                txt=''
            txt = txt+book[0][1]
            lab_tags[book[1]] = txt
            prelab = lab

In [9]:
filterwords=[
    '年','月','中央社','記者','編輯','譯者','日電',
    '\n',' ','「','」'
    ,'今天','說','表示','年','月','日','後','前','時','應','中','今年'
    ,'兩','下','人','最','次','再','先','盡量'
]

In [10]:
articles = []
titles = []
for news in newslist:
    articles.append((news[0][1],news[1]))
    titles.append((news[0][0],news[1]))

In [11]:
books = []

n = 0
while n < len(titles):
    head_dic = {}
    txt_dic = {}
    head = titles[n]
    txt = articles[n]
    
    head_dic[head[0]] = head[1]
    txt_dic[txt[0]] = txt[1]
    books.append([head_dic,txt_dic])
    n += 1

In [12]:
def get_tags(books=books):

    prelab = ''
    tags = {}
    for book in books:
        head_dic = book[0]
        txt_dic = book[1]
        n=0
        for dic in [head_dic,txt_dic]:

            for lab in ['生活','運動','政治','科技','國際','兩岸','產經','社會','地方','娛樂','證券','文化']:
                if list(dic.values())[0]==lab:
                    if prelab != lab:
                        txt=''
                    txt += list(dic.keys())[0]
                    tags[list(dic.values())[0]] = txt
                    prelab = lab
            if n == 0:
                title_tags = tags
            elif n == 1:
                txt_tags = tags
            n += 1
    return title_tags,txt_tags

In [13]:
title_tags,txt_tags = get_tags(books)

In [14]:
title_tags

{'兩岸': '藍營縣市長訪陸  賴峰偉：我就是將來他們的模式國民黨籍澎湖縣長當選人賴峰偉昨天率團訪問北京並會見國台辦主任劉結一。賴峰偉今天受訪時說，未來國民黨籍縣市長登陸交流，「我這個模式就是將來他們來訪的模式」。賴峰偉和部分團員今天在下榻飯店接受台灣媒體的訪問時指出，國民黨在這次九合一選舉中拿下15縣市，訪問團昨天與劉結一見面時談到，未來藍營縣市長只要到大陸來爭取對地方有利的事項時，大陸都會「積極回應」。他說，「我這個模式就是將來他們來訪的模式」。賴峰偉昨天還提出觀光直航等問題，包括麗星郵輪復航、開闢上海班機直航以及開通東山島客貨航線，另外還談到約束大陸越界捕魚等議題。賴峰偉說，劉結一回應表示會積極辦理澎湖提出的事情，「而且儘量朝向對澎湖有利的商機前進」。另外，大陸已指派專門的窗口，讓澎湖「隨時反應事情」。賴峰偉等人今天將結束訪問返回澎湖。對於外傳陸委會請託他向劉結一傳話一事，賴峰偉低調表示「這個就不必再提」，並指國台辦也知道這件事；而陸委會請託傳話的事「是茶餘飯後大家在聊，也不是很正式」。已拿到當選證書、但尚未就職的賴峰偉是九合一選後台灣首位登陸的準縣市長。對此，他表示，這次來北京是剛好因緣際會，是無黨聯盟主席林炳坤牽線才組團訪陸，並非外界定義的「搶頭香」。體育活動交流也是賴峰會等人這次訪問北京的一項重點。訪問團成員、澎湖縣副議長陳雙全說，澎湖明年將舉辦2020東京奧運會前賽，希望大陸選手也共襄盛舉。他表示，澎湖將爭取2022年風帆世界錦標賽，這兩天與大陸帆船體育總會等相關單位做了溝通，希望未來有更多大陸選手到澎湖參加體育交流和比賽。陳雙全指出，拜會中還向大陸國台辦提出例如桌球、羽毛球等後續兩岸體育活動交流活動，國台辦表示會盡力配合。學者：陸民間獵巫情緒高  吳寶春事件北京難收拾學者王智盛今天分析，吳寶春事件顯示，北京長期以民族主義操作對台政策，導致民間獵巫情緒高漲，難以收拾，只能任憑「尾巴搖狗」。這勢必引起台灣民意反感，兩岸更加對立。吳寶春麵包店近日將在中國上海開幕，卻因吳寶春過去受訪時表示「不會把眼光只看在中國」被中國網民打為「台獨麵包」。對此，吳寶春昨天在「吳寶春麥方店-高雄店」官方臉書發聲明強調，「身為中國人，是我的驕傲。『兩岸一家親』是我堅持不變的態度。」並表態支持「九二共識」。兩岸政策協會秘書長王智盛下午接受中央社記者訪問時表示，這次事件與85度C

### 將title_tags，txt_tags中全文字斷詞為tokens，統計tokens找出高頻字整理為lab_fwords

In [15]:
def get_fwords(tags={},top=300):


    lab_fwords={}
#    lab_Ftwowords = {}
    for labname in tags.keys():
        print(labname,end=',')
        lab_words = tags[labname]
        seg_list = jieba.cut(lab_words, cut_all=False)
        txt = ",".join(seg_list)
        tokens = txt.split(',')
        tokens = list(filter(lambda a: a not in stopwords and a not in filterwords and a.isalpha() == True,tokens))#and len(a)>1, tokens))
        lab_alltwowords = list(nltk.bigrams(tokens))
        two_tokens =  [(i[0]+i[1]) for i in lab_alltwowords]
        tokens.extend(two_tokens)
        lab_fwords[labname] =  [i[0] for i in nltk.FreqDist(tokens).most_common(top)]
    return lab_fwords

In [16]:
txt_fwords3 =get_fwords(txt_tags)
txt_fwords3

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\gr11n\AppData\Local\Temp\jieba.cache


政治,

Loading model cost 1.179 seconds.
Prefix dict has been built succesfully.


證券,生活,文化,運動,兩岸,社會,產經,國際,地方,娛樂,科技,

{'兩岸': ['中國',
  '報導',
  '香港',
  '大陸',
  '台灣',
  '兩岸',
  '經濟',
  '已',
  '企業',
  '中共',
  '論壇',
  '北京',
  '政府',
  '人員',
  '指出',
  '合作',
  '名',
  '上海',
  '工作',
  '發展',
  '國家',
  '要求',
  '問題',
  '相關',
  '代表',
  '軍人',
  '社會',
  '發生',
  '進行',
  '中國大陸',
  '美國',
  '退役',
  '事件',
  '遭',
  '包括',
  '政策',
  '政治',
  '中央',
  '改革',
  '委員會',
  '當局',
  '退役軍人',
  '泛民',
  '劉曉波',
  '元',
  '媒體',
  '官方',
  '曾',
  '雙城',
  '就業',
  '雙城論壇',
  '交流',
  '新',
  '疫情',
  '人士',
  '憲法',
  '公司',
  '犯罪',
  '議員',
  '新聞',
  '民眾',
  '網路',
  '新疆',
  '吳寶春',
  '種',
  '自由',
  '措施',
  '憲章',
  '陸委會',
  '支持',
  '官員',
  '主任',
  '提出',
  '認為',
  '指',
  '透過',
  '目前',
  '警方',
  '民主',
  '立法會',
  '強調',
  '方面',
  '習近平',
  '組織',
  '台',
  '首',
  '來台',
  '宗教',
  '教育',
  '主要',
  '零八',
  '零八憲章',
  '公布',
  '發布',
  '城市',
  '選舉',
  '地方',
  '市場',
  '當地',
  '未',
  '發表',
  '情況',
  '分',
  '活動',
  '期間',
  '人民幣',
  '全國',
  '文化',
  '部',
  '非洲',
  '豬瘟',
  '非洲豬瘟',
  '大學',
  '宣傳',
  '些',
  '以來',
  '萬人',
  '文章',
  '當時',
  '人口',
  '產業',
  '昨天',
  '卻',
  '出席',


### 定義remove_mix(lab_fwords)，刪除交集在各類別中的高頻字，加強高頻字辨識度

In [17]:
#去除交集的高頻字
def remove_mix(t):
    lab_fwords = t

    dic3={}
    for lab in lab_fwords.keys():   
        word_content2 = lab_fwords[lab].copy()
        for lab2 in lab_fwords.keys():        
            if lab != lab2:
                word_content = [w for w in lab_fwords[lab] if (w in [a for a in lab_fwords[lab2]])]
                for w in word_content:
                    try:
                        word_content2.remove(w)                  
                    except ValueError:
                        pass
        dic3[lab] = word_content2        
    return dic3

In [18]:
title_fwords = remove_mix(get_fwords(title_tags,700))
txt_fwords = remove_mix(get_fwords(txt_tags,800))

政治,證券,生活,文化,運動,兩岸,社會,產經,國際,地方,娛樂,科技,政治,證券,生活,文化,運動,兩岸,社會,產經,國際,地方,娛樂,科技,

In [19]:
import random
random.shuffle(books)

## 3. 抽取特徵

### 定義pos_fwords()，找出文章中出現在各類別中的高頻字

pos_fwords(article,title=False,top=200)
- article = str，目標文本
- title = bool，文本是否是標題
- top = int，選取前幾名高頻字進行特徵抽取
```
output = {
    word1 : True,
    word2 : True,
    word3 : True,
    ...
    }
 ```

In [20]:
def pos_fwords(article,title=False,top=200):
    if title==True:
        fwords = title_fwords
    elif title==False:
        fwords = txt_fwords
    else:
        print('title=True or False')
    features={}
    t = ''
    seg_list = jieba.cut_for_search(article)
    t = ",".join(seg_list)
    article_tokens = t.split(',')
    article_tokens = [w for w in article_tokens if w.isalpha()]

    filt_tokens = list(filter(lambda a: a not in stopwords and a not in filterwords , article_tokens))

    toks = [w[0] for w in nltk.FreqDist(filt_tokens).most_common(top)]


    if len([i for i in toks if i in [w for li in fwords.values() for w in li]])==0 :
        for i in toks[:int(top)]:
            features[i] = True
    else:
        bigcontent=[]
        kk=''
        for k in fwords.keys():
            content=[]
            for tok in toks:
                if tok in fwords[k] :
                    content.append(tok)
                if len(bigcontent) < len(content):
                    bigcontent = content.copy()
                    kk = k

        for tok in bigcontent:  
            features[tok] = True
    if len(list(features.keys())) < 3:
        features={}
    return features

In [22]:
#pos_fwords(test_txt,title=False,top=200)

### 定義特徵抽取器get_features()，抽出文章特徵
get_features(dic={},title=False,top=200)

- dic = dic，{ 字串 : 類別 }
- title = bool，字串是否為標題
- top = int，選取前幾名高頻字進行特徵抽取

```
output = [ pos_fwords() , 類別 ]

```

In [23]:
def get_features(dic={},title=False,top=200):
    n = 0
#    head_dic = book[0]
    article = list(dic.keys())[0]
    labname = list(dic.values())[0]
    features = pos_fwords(article,title,top)

    if len(list(features.keys())) < 2:
        feature_item = [{},labname]
    else:
        feature_item = [features,labname]

    return feature_item

### 定義get_featuresets()，將分類好的文本集books由特徵抽取器get_features()做成機器學習接受格式的特徵集
```
output=[
    [ pos_fwords(文本1) , 類別 ],
    [ pos_fwords(文本2) , 類別 ],
    [ pos_fwords(文本3) , 類別 ],
    ...
]
```

In [26]:
def get_featuresets(books=books,top=200):
    title_featuresets = []
    txt_featuresets = []
    for book in books:

        title_featureset = get_features(book[0],title=True,top=top)
        title_featuresets.append(title_featureset)
        
        txt_featureset = get_features(book[1],title=False,top=top)
        txt_featuresets.append(txt_featureset)
    return title_featuresets, txt_featuresets

## 4. 使用特徵集進行機器學習

### 製作標題與內文2份特徵集，分別以7:3的比例作為訓練集和測試集，最後使用貝氏分類器製作模型並回傳精確度

In [27]:
title_featuresets, txt_featuresets = get_featuresets(top=170)

In [28]:
title_featuresets = title_featuresets.copy()
size = int(len(title_featuresets)*0.7)
train_set, test_set = title_featuresets[:size], title_featuresets[size:]
title_classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(title_classifier, test_set)

0.32222222222222224

In [29]:
txt_featuresets = txt_featuresets.copy()
size = int(len(txt_featuresets)*0.7)
train_set, test_set = txt_featuresets[:size], txt_featuresets[size:]
txt_classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(txt_classifier, test_set)

0.8472222222222222

### 使用txt_classifier.classify_many()觀察測試結果，並與測試集進行比對

In [30]:
title_comanswer = title_classifier.classify_many([i[0] for i in title_featuresets])
txt_comanswer = txt_classifier.classify_many([i[0] for i in txt_featuresets])

title_answer = [i[1] for i in title_featuresets]
txt_answer = [i[1] for i in txt_featuresets]

In [31]:
import pandas as pd
import numpy as np

In [32]:
title_correct = ['F' if w != title_answer[title_comanswer.index(w)] else 'T' for w in title_comanswer ]
txt_correct = ['F' if w != txt_answer[txt_comanswer.index(w)] else 'T' for w in txt_comanswer ]
df = pd.DataFrame([txt_comanswer,txt_answer,txt_correct,title_comanswer,title_answer,title_correct],index=['txt_comanswer','txt_answer','txt_correct','title_comanswer','title_answer','title_correct']).T

In [33]:
df.head(10)

,txt_comanswer,txt_answer,txt_correct,title_comanswer,title_answer,title_correct
0,娛樂,娛樂,T,娛樂,娛樂,T
1,兩岸,兩岸,T,兩岸,兩岸,T
2,兩岸,兩岸,T,兩岸,兩岸,T
3,社會,社會,T,社會,社會,T
4,生活,科技,F,兩岸,科技,T
5,運動,運動,T,運動,運動,T
6,證券,證券,T,兩岸,證券,T
7,娛樂,娛樂,T,兩岸,娛樂,T
8,兩岸,兩岸,T,兩岸,兩岸,T
9,娛樂,娛樂,T,娛樂,娛樂,T


## 5. 使用訓練結果再做一次機器學習

### 由於依照標題辨識類別的精確度太低，另外加入新的特徵
### - 定義get_top()統計文本高頻字出現最多次的類別
```
get_top(article,top,get=20)=
- article = str，目標文本
- top = int，高頻字數量
- get = int，統計前幾名高頻字類別

output = 類別
```

In [36]:
from collections import Counter
def get_top(article,top,get=20):
#    print(article)
    features={}
    t = ''
    seg_list = jieba.cut_for_search(article)
    t = ",".join(seg_list)
    article_tokens = t.split(',')
    article_tokens = [w for w in article_tokens if w.isalpha()]

    filt_tokens = list(filter(lambda a: a not in stopwords and a not in filterwords , article_tokens))
    toks = [w for w in nltk.FreqDist(filt_tokens).most_common(top)]
    txt_comanwer = txt_classifier.classify(pos_fwords(article,title=False))

    try:
        selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords.values() and t[1]>=1 for w in li]])
        klist=[]
        for w in selectwords[:get]:  
            for k in txt_fwords.keys():
                if w[0] in [s for s in txt_fwords[k]] :
#                    print([w[0],k])
                    klist.append(k)
#        print("")
        one = Counter(klist).most_common(1)[0][0]
        two = Counter(klist).most_common(2)[1][0]
        count1 = Counter(klist).most_common(1)[0][1]
        count2 = Counter(klist).most_common(2)[1][1]
        X = int((count1+count2)/6)
        if X < 1:
            X == 1
        
        if Counter(klist).most_common(1)[0][1] - Counter(klist).most_common(2)[1][1] <=X and (one == txt_comanwer or two == txt_comanwer):
            features['top'] = txt_comanwer
        else:
            features['top'] = Counter(klist).most_common(1)[0][0]
    except:
        try:
            selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords.values()  for w in li]])
            klist=[]
            for w in selectwords[:get]:  
                for k in txt_fwords.keys():
                    if w[0] in [s for s in txt_fwords[k]]  and w[1]>=1 :
#                        print([w[0],k])
                        klist.append(k)
#            print("")
            one = Counter(klist).most_common(1)[0][0]
            two = Counter(klist).most_common(2)[1][0]
            count1 = Counter(klist).most_common(1)[0][1]
            count2 = Counter(klist).most_common(2)[1][1]
            X = int((count1+count2)/6)
            if X < 1:
                X == 1
            print(Counter(klist).most_common(5))
            if Counter(klist).most_common(1)[0][1] - Counter(klist).most_common(2)[1][1] <=X and (one == txt_comanwer or two == txt_comanwer):
                features['top'] = txt_comanwer
            else:
                features['top'] = Counter(klist).most_common(1)[0][0]
        except IndexError:
            try:
                selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords3.values() for w in li]])
                klist=[]
                for w in selectwords[:get]:  
                    for k in txt_fwords3.keys():
                        if w[0] in [s for s in txt_fwords3[k]] and w[1]>=3 :
                            klist.append(k)
#                        print([w[0],k])
#                print("")
                one = Counter(klist).most_common(1)[0][0]
                two = Counter(klist).most_common(2)[1][0]
                count1 = Counter(klist).most_common(1)[0][1]
                count2 = Counter(klist).most_common(2)[1][1]
                X = int((count1+count2)/6)
                if X < 1:
                    X == 1
                if Counter(klist).most_common(1)[0][1] - Counter(klist).most_common(2)[1][1] <=X and (one == txt_comanwer or two == txt_comanwer):
                    features['top'] = txt_comanwer
                else:
                    features['top'] = Counter(klist).most_common(1)[0][0]
            except :
                selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords3.values() for w in li]])
                klist=[]
                for w in selectwords[:get]:  
                    for k in txt_fwords3.keys():
                        if w[0] in [s for s in txt_fwords3[k]]  :
                            klist.append(k)
 #                       print([w[0],k])
 #               print("")
                one = Counter(klist).most_common(1)[0][0]
                two = Counter(klist).most_common(2)[1][0]
                count1 = Counter(klist).most_common(1)[0][1]
                count2 = Counter(klist).most_common(2)[1][1]
                X = int((count1+count2)/6)
                if X < 1:
                    X == 1
                if Counter(klist).most_common(1)[0][1] - Counter(klist).most_common(2)[1][1] <=X and (one == txt_comanwer or two == txt_comanwer):
                    features['top'] = txt_comanwer
                else:
                    features['top'] = Counter(klist).most_common(1)[0][0]
    
#    print(['one='+one,'two='+two,'txt_comanwer='+txt_comanwer])
    print('txt_comanwer='+txt_comanwer)
    print(features.values())
    print("")
    return features

In [37]:
txt_top = [list(get_top(txt,175,175).values())[0] for txt in [list(book[1].keys())[0] for book in books]]

[('娛樂', 11), ('地方', 1), ('產經', 1), ('國際', 1), ('文化', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('兩岸', 7), ('社會', 2), ('生活', 1), ('科技', 1), ('國際', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

[('兩岸', 5), ('政治', 4), ('娛樂', 1), ('產經', 1), ('生活', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

[('社會', 5), ('地方', 3), ('運動', 1), ('國際', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('科技', 3), ('娛樂', 3), ('生活', 3), ('國際', 3), ('產經', 2)]
txt_comanwer=生活
dict_values(['科技'])

[('運動', 24), ('地方', 3), ('生活', 3), ('證券', 2), ('產經', 2)]
txt_comanwer=運動
dict_values(['運動'])

[('證券', 6), ('娛樂', 2), ('生活', 2), ('科技', 2), ('產經', 1)]
txt_comanwer=證券
dict_values(['證券'])

[('娛樂', 12), ('運動', 3), ('科技', 1), ('兩岸', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('兩岸', 4), ('政治', 2), ('證券', 2), ('文化', 1), ('產經', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

[('娛樂', 4), ('國際', 3), ('地方', 2), ('產經', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('運動', 7), ('娛樂', 1), ('政治', 1), ('地方', 1), ('科技', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('文化',

[('娛樂', 13), ('運動', 3), ('文化', 3), ('政治', 2), ('地方', 2)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('運動', 20), ('文化', 3), ('產經', 2), ('地方', 2), ('社會', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('地方', 6), ('文化', 3), ('生活', 2), ('兩岸', 2), ('科技', 2)]
txt_comanwer=地方
dict_values(['地方'])

[('娛樂', 13), ('文化', 3), ('科技', 2), ('證券', 2), ('政治', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('運動', 20), ('政治', 2), ('娛樂', 2), ('文化', 1), ('產經', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('娛樂', 9), ('社會', 4), ('文化', 1), ('證券', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('娛樂', 13), ('政治', 7), ('社會', 3), ('運動', 3), ('文化', 3)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('證券', 22), ('科技', 3), ('兩岸', 2), ('產經', 2), ('文化', 1)]
txt_comanwer=證券
dict_values(['證券'])

[('證券', 40), ('產經', 2), ('兩岸', 1), ('社會', 1)]
txt_comanwer=證券
dict_values(['證券'])

[('科技', 19), ('國際', 4), ('社會', 4), ('證券', 1), ('生活', 1)]
txt_comanwer=科技
dict_values(['科技'])

[('社會', 9), ('地方', 1), ('運動', 1), ('科技', 1)]
txt_comanwer=社會
dict_values(['社會'])

[(

[('證券', 3), ('科技', 3), ('生活', 2), ('社會', 2), ('兩岸', 1)]
txt_comanwer=證券
dict_values(['證券'])

[('產經', 14), ('政治', 3), ('生活', 2), ('社會', 1), ('地方', 1)]
txt_comanwer=產經
dict_values(['產經'])

[('兩岸', 13), ('娛樂', 2), ('文化', 2), ('證券', 2), ('科技', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

[('科技', 5), ('國際', 3), ('地方', 2), ('生活', 2), ('產經', 1)]
txt_comanwer=科技
dict_values(['科技'])

[('產經', 3), ('國際', 3), ('證券', 3), ('生活', 2), ('兩岸', 2)]
txt_comanwer=證券
dict_values(['產經'])

txt_comanwer=證券
dict_values(['證券'])

[('娛樂', 9), ('運動', 4), ('科技', 3), ('文化', 2), ('社會', 2)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('娛樂', 11), ('文化', 3), ('兩岸', 2), ('運動', 2), ('證券', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('國際', 12), ('政治', 3), ('地方', 1), ('產經', 1), ('證券', 1)]
txt_comanwer=國際
dict_values(['國際'])

[('文化', 9), ('社會', 2), ('娛樂', 2), ('生活', 1), ('政治', 1)]
txt_comanwer=文化
dict_values(['文化'])

[('國際', 26), ('生活', 1), ('運動', 1), ('地方', 1), ('科技', 1)]
txt_comanwer=國際
dict_values(['國際'])

[('證券', 5), ('產經', 2)]
txt_c

[('社會', 27), ('科技', 1), ('國際', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('地方', 12), ('產經', 4), ('政治', 3), ('運動', 3), ('證券', 2)]
txt_comanwer=地方
dict_values(['地方'])

[('兩岸', 6), ('社會', 4), ('文化', 2), ('科技', 2), ('國際', 2)]
txt_comanwer=兩岸
dict_values(['兩岸'])

[('政治', 6), ('運動', 2), ('地方', 1)]
txt_comanwer=政治
dict_values(['政治'])

[('兩岸', 6), ('科技', 4), ('社會', 1), ('國際', 1), ('生活', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

[('兩岸', 8), ('政治', 3), ('運動', 3), ('國際', 2), ('證券', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

[('政治', 5), ('社會', 3), ('地方', 2), ('兩岸', 2), ('運動', 1)]
txt_comanwer=政治
dict_values(['政治'])

[('科技', 3), ('產經', 3), ('國際', 3), ('生活', 2), ('政治', 2)]
txt_comanwer=生活
dict_values(['科技'])

[('政治', 12), ('文化', 4), ('娛樂', 4), ('地方', 2), ('國際', 1)]
txt_comanwer=政治
dict_values(['政治'])

[('證券', 7), ('產經', 3)]
txt_comanwer=證券
dict_values(['證券'])

[('地方', 7), ('社會', 4), ('政治', 3), ('證券', 1), ('運動', 1)]
txt_comanwer=地方
dict_values(['地方'])

[('國際', 10), ('兩岸', 2), ('社會', 2), ('政治', 2), ('娛樂'

[('娛樂', 5), ('生活', 3), ('文化', 2), ('社會', 2), ('兩岸', 2)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('產經', 10), ('證券', 4), ('政治', 2), ('科技', 2), ('生活', 2)]
txt_comanwer=產經
dict_values(['產經'])

[('科技', 22), ('兩岸', 3), ('國際', 2), ('社會', 1), ('地方', 1)]
txt_comanwer=科技
dict_values(['科技'])

[('娛樂', 5), ('社會', 3), ('生活', 2), ('政治', 2), ('科技', 2)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('證券', 24), ('兩岸', 3), ('運動', 2), ('產經', 1), ('政治', 1)]
txt_comanwer=證券
dict_values(['證券'])

[('社會', 24), ('文化', 1), ('地方', 1), ('運動', 1), ('證券', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('證券', 13), ('國際', 4), ('兩岸', 3), ('科技', 3), ('產經', 2)]
txt_comanwer=證券
dict_values(['證券'])

[('科技', 7), ('國際', 2), ('運動', 2), ('兩岸', 1), ('證券', 1)]
txt_comanwer=科技
dict_values(['科技'])

[('娛樂', 10), ('運動', 2), ('生活', 2), ('社會', 1), ('文化', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('產經', 5), ('科技', 3), ('國際', 3), ('文化', 2), ('證券', 2)]
txt_comanwer=產經
dict_values(['產經'])

[('地方', 4), ('兩岸', 2), ('政治', 2), ('文化', 1), ('科技', 1)]
txt_coma

...

[('社會', 13), ('證券', 1), ('生活', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('科技', 5), ('證券', 1)]
txt_comanwer=科技
dict_values(['科技'])

[('運動', 23), ('政治', 2), ('娛樂', 1), ('文化', 1), ('科技', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('證券', 9), ('科技', 5), ('產經', 3), ('國際', 2), ('兩岸', 1)]
txt_comanwer=證券
dict_values(['證券'])

[('運動', 12), ('娛樂', 9), ('文化', 7), ('兩岸', 2), ('國際', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('國際', 7), ('生活', 3), ('證券', 2), ('兩岸', 2), ('科技', 2)]
txt_comanwer=國際
dict_values(['國際'])

[('政治', 4), ('地方', 3), ('證券', 2), ('產經', 1), ('娛樂', 1)]
txt_comanwer=政治
dict_values(['政治'])

[('文化', 15), ('運動', 3), ('娛樂', 2), ('社會', 1), ('產經', 1)]
txt_comanwer=文化
dict_values(['文化'])

[('娛樂', 30), ('運動', 2), ('社會', 1), ('文化', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('科技', 8), ('文化', 2), ('兩岸', 2), ('產經', 2), ('證券', 1)]
txt_comanwer=科技
dict_values(['科技'])

[('社會', 11), ('科技', 3), ('文化', 2), ('生活', 1), ('國際', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('運動', 35), ('社會', 2), ('娛樂', 2), ('

[('運動', 52), ('社會', 1), ('文化', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('社會', 2), ('生活', 1), ('科技', 1)]
txt_comanwer=兩岸
dict_values(['社會'])

[('運動', 22), ('生活', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('運動', 41), ('社會', 2), ('文化', 1), ('政治', 1), ('地方', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('社會', 7), ('產經', 1), ('運動', 1), ('國際', 1), ('地方', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('科技', 8), ('國際', 2), ('社會', 2), ('文化', 1), ('兩岸', 1)]
txt_comanwer=科技
dict_values(['科技'])

[('社會', 16), ('政治', 3), ('生活', 1), ('文化', 1), ('科技', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('科技', 2), ('國際', 1)]
txt_comanwer=兩岸
dict_values(['科技'])

[('國際', 8), ('證券', 2), ('文化', 1), ('運動', 1), ('生活', 1)]
txt_comanwer=國際
dict_values(['國際'])

[('文化', 4), ('娛樂', 3), ('科技', 1), ('社會', 1), ('證券', 1)]
txt_comanwer=文化
dict_values(['文化'])

[('地方', 3), ('文化', 3), ('娛樂', 3), ('社會', 2), ('生活', 1)]
txt_comanwer=地方
dict_values(['地方'])

txt_comanwer=社會
dict_values(['社會'])

[('社會', 17), ('地方', 2), ('國際', 1), ('兩岸', 1)]


[('國際', 12), ('兩岸', 3), ('運動', 3), ('娛樂', 1), ('政治', 1)]
txt_comanwer=國際
dict_values(['國際'])

[('國際', 5), ('科技', 3), ('兩岸', 2), ('產經', 1), ('社會', 1)]
txt_comanwer=國際
dict_values(['國際'])

[('娛樂', 9), ('文化', 5), ('證券', 2), ('兩岸', 2), ('國際', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('運動', 45), ('娛樂', 2), ('地方', 1), ('科技', 1), ('文化', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('地方', 9), ('政治', 3), ('文化', 2), ('生活', 2), ('社會', 1)]
txt_comanwer=地方
dict_values(['地方'])

[('文化', 4), ('國際', 2), ('運動', 1), ('政治', 1)]
txt_comanwer=文化
dict_values(['文化'])

[('國際', 5), ('兩岸', 4), ('政治', 2), ('運動', 2), ('產經', 2)]
txt_comanwer=國際
dict_values(['國際'])

[('娛樂', 11), ('國際', 2), ('社會', 2), ('產經', 1), ('運動', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('地方', 9), ('運動', 7), ('文化', 4), ('社會', 2), ('兩岸', 1)]
txt_comanwer=地方
dict_values(['地方'])

[('社會', 13), ('地方', 2), ('文化', 1), ('兩岸', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('科技', 9), ('兩岸', 2), ('政治', 1), ('娛樂', 1), ('證券', 1)]
txt_comanwer=科技
dict_values(['科技

[('娛樂', 8), ('國際', 8), ('科技', 3), ('社會', 2), ('政治', 1)]
txt_comanwer=國際
dict_values(['國際'])

[('政治', 10), ('娛樂', 3), ('兩岸', 2), ('國際', 2), ('地方', 2)]
txt_comanwer=政治
dict_values(['政治'])

[('文化', 14), ('兩岸', 1), ('地方', 1), ('政治', 1), ('證券', 1)]
txt_comanwer=文化
dict_values(['文化'])

[('政治', 7), ('社會', 4), ('科技', 1), ('地方', 1)]
txt_comanwer=政治
dict_values(['政治'])

[('證券', 11), ('產經', 10), ('生活', 2), ('文化', 2), ('政治', 2)]
txt_comanwer=證券
dict_values(['證券'])

[('娛樂', 20), ('文化', 2), ('政治', 2), ('兩岸', 1), ('科技', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

[('政治', 10), ('國際', 3), ('地方', 1), ('產經', 1), ('社會', 1)]
txt_comanwer=政治
dict_values(['政治'])

[('科技', 29), ('產經', 3), ('社會', 1), ('地方', 1), ('兩岸', 1)]
txt_comanwer=科技
dict_values(['科技'])

[('運動', 25), ('國際', 3), ('娛樂', 3), ('產經', 2), ('政治', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('地方', 10), ('社會', 5), ('產經', 2), ('生活', 1), ('證券', 1)]
txt_comanwer=地方
dict_values(['地方'])

[('生活', 18), ('社會', 5), ('證券', 2), ('運動', 1), ('地方', 1)]
txt_comanwer=生活

[('社會', 15), ('科技', 4), ('生活', 3), ('產經', 2), ('政治', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('文化', 7), ('科技', 4), ('娛樂', 1), ('國際', 1), ('運動', 1)]
txt_comanwer=文化
dict_values(['文化'])

[('兩岸', 4), ('政治', 3), ('生活', 2), ('地方', 2), ('娛樂', 2)]
txt_comanwer=兩岸
dict_values(['兩岸'])

[('運動', 13), ('政治', 1), ('地方', 1), ('生活', 1), ('娛樂', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('兩岸', 14), ('國際', 4), ('政治', 2), ('地方', 2), ('文化', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

[('文化', 5), ('社會', 4), ('國際', 3), ('兩岸', 2), ('運動', 1)]
txt_comanwer=文化
dict_values(['文化'])

[('社會', 8), ('地方', 5), ('政治', 3), ('運動', 2), ('科技', 1)]
txt_comanwer=社會
dict_values(['社會'])

[('生活', 8), ('產經', 2), ('地方', 1), ('政治', 1), ('科技', 1)]
txt_comanwer=生活
dict_values(['生活'])

[('生活', 10), ('地方', 2), ('政治', 1), ('國際', 1)]
txt_comanwer=生活
dict_values(['生活'])

[('運動', 39), ('地方', 1), ('證券', 1), ('生活', 1)]
txt_comanwer=運動
dict_values(['運動'])

[('地方', 4), ('社會', 1), ('兩岸', 1), ('科技', 1), ('國際', 1)]
txt_comanwer=地方
dict_values(['地

### 製作新的特徵集dataset
```
dataset=[
    ({'title_comanswer': 預測類別, 'txt_comanswer': 預測類別, 'txt_top': 預測類別}, 正確類別),
    ({'title_comanswer': 預測類別, 'txt_comanswer': 預測類別, 'txt_top': 預測類別}, 正確類別),
    ({'title_comanswer': 預測類別, 'txt_comanswer': 預測類別, 'txt_top': 預測類別}, 正確類別),
    ...   
]
```

### 使用Sklearn做貝氏模型與SVM模型，並得出精確度

In [38]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

In [39]:
df2 = pd.DataFrame([txt_comanswer,txt_top,txt_answer,txt_correct,title_comanswer,title_answer,title_correct],index=['txt_comanswer','txt_top','txt_answer','txt_correct','title_comanswer','title_answer','title_correct']).T

In [40]:
dataset = []

answer = ""
for i,v in df2.iterrows():
    answer_dict={}
    print(v)
#    answer_dict['txt_comanswer']=v[0]
    answer_dict['txt_top']=v[1]
#    answer_dict['title_comanswer']=v[4]

    answer = v[1]
    dataset.append((answer_dict,answer))

txt_comanswer      娛樂
txt_top            娛樂
txt_answer         娛樂
txt_correct         T
title_comanswer    娛樂
title_answer       娛樂
title_correct       T
Name: 0, dtype: object
txt_comanswer      兩岸
txt_top            兩岸
txt_answer         兩岸
txt_correct         T
title_comanswer    兩岸
title_answer       兩岸
title_correct       T
Name: 1, dtype: object
txt_comanswer      兩岸
txt_top            兩岸
txt_answer         兩岸
txt_correct         T
title_comanswer    兩岸
title_answer       兩岸
title_correct       T
Name: 2, dtype: object
txt_comanswer      社會
txt_top            社會
txt_answer         社會
txt_correct         T
title_comanswer    社會
title_answer       社會
title_correct       T
Name: 3, dtype: object
txt_comanswer      生活
txt_top            科技
txt_answer         科技
txt_correct         F
title_comanswer    兩岸
title_answer       科技
title_correct       T
Name: 4, dtype: object
txt_comanswer      運動
txt_top            運動
txt_answer         運動
txt_correct         T
title_comanswer    運動
title

Name: 104, dtype: object
txt_comanswer      政治
txt_top            政治
txt_answer         政治
txt_correct         T
title_comanswer    政治
title_answer       政治
title_correct       T
Name: 105, dtype: object
txt_comanswer      地方
txt_top            地方
txt_answer         地方
txt_correct         F
title_comanswer    兩岸
title_answer       地方
title_correct       T
Name: 106, dtype: object
txt_comanswer      娛樂
txt_top            娛樂
txt_answer         娛樂
txt_correct         T
title_comanswer    兩岸
title_answer       娛樂
title_correct       T
Name: 107, dtype: object
txt_comanswer      社會
txt_top            社會
txt_answer         社會
txt_correct         T
title_comanswer    社會
title_answer       社會
title_correct       T
Name: 108, dtype: object
txt_comanswer      文化
txt_top            文化
txt_answer         文化
txt_correct         F
title_comanswer    兩岸
title_answer       文化
title_correct       T
Name: 109, dtype: object
txt_comanswer      產經
txt_top            產經
txt_answer         產經
txt_correct   

txt_comanswer      文化
txt_top            文化
txt_answer         文化
txt_correct         F
title_comanswer    兩岸
title_answer       文化
title_correct       T
Name: 212, dtype: object
txt_comanswer      生活
txt_top            生活
txt_answer         生活
txt_correct         F
title_comanswer    生活
title_answer       生活
title_correct       T
Name: 213, dtype: object
txt_comanswer      科技
txt_top            科技
txt_answer         科技
txt_correct         T
title_comanswer    兩岸
title_answer       科技
title_correct       T
Name: 214, dtype: object
txt_comanswer      科技
txt_top            科技
txt_answer         科技
txt_correct         T
title_comanswer    科技
title_answer       科技
title_correct       T
Name: 215, dtype: object
txt_comanswer      娛樂
txt_top            娛樂
txt_answer         娛樂
txt_correct         T
title_comanswer    兩岸
title_answer       娛樂
title_correct       T
Name: 216, dtype: object
txt_comanswer      文化
txt_top            文化
txt_answer         文化
txt_correct         F
title_comanswer  

Name: 317, dtype: object
txt_comanswer      兩岸
txt_top            生活
txt_answer         生活
txt_correct         T
title_comanswer    兩岸
title_answer       生活
title_correct       T
Name: 318, dtype: object
txt_comanswer      社會
txt_top            社會
txt_answer         社會
txt_correct         T
title_comanswer    兩岸
title_answer       社會
title_correct       T
Name: 319, dtype: object
txt_comanswer      娛樂
txt_top            娛樂
txt_answer         娛樂
txt_correct         T
title_comanswer    娛樂
title_answer       娛樂
title_correct       T
Name: 320, dtype: object
txt_comanswer      地方
txt_top            地方
txt_answer         地方
txt_correct         F
title_comanswer    地方
title_answer       地方
title_correct       T
Name: 321, dtype: object
txt_comanswer      證券
txt_top            證券
txt_answer         證券
txt_correct         T
title_comanswer    證券
title_answer       證券
title_correct       T
Name: 322, dtype: object
txt_comanswer      娛樂
txt_top            娛樂
txt_answer         娛樂
txt_correct   

txt_comanswer      科技
txt_top            科技
txt_answer         產經
txt_correct         T
title_comanswer    產經
title_answer       產經
title_correct       T
Name: 426, dtype: object
txt_comanswer      政治
txt_top            政治
txt_answer         政治
txt_correct         T
title_comanswer    政治
title_answer       政治
title_correct       T
Name: 427, dtype: object
txt_comanswer      國際
txt_top            國際
txt_answer         國際
txt_correct         T
title_comanswer    國際
title_answer       國際
title_correct       T
Name: 428, dtype: object
txt_comanswer      生活
txt_top            生活
txt_answer         生活
txt_correct         F
title_comanswer    兩岸
title_answer       生活
title_correct       T
Name: 429, dtype: object
txt_comanswer      社會
txt_top            社會
txt_answer         社會
txt_correct         T
title_comanswer    兩岸
title_answer       社會
title_correct       T
Name: 430, dtype: object
txt_comanswer      科技
txt_top            科技
txt_answer         科技
txt_correct         T
title_comanswer  

...

txt_comanswer      兩岸
txt_top            兩岸
txt_answer         兩岸
txt_correct         T
title_comanswer    兩岸
title_answer       兩岸
title_correct       T
Name: 659, dtype: object
txt_comanswer      產經
txt_top            產經
txt_answer         科技
txt_correct         T
title_comanswer    兩岸
title_answer       科技
title_correct       T
Name: 660, dtype: object
txt_comanswer      兩岸
txt_top            兩岸
txt_answer         兩岸
txt_correct         T
title_comanswer    兩岸
title_answer       兩岸
title_correct       T
Name: 661, dtype: object
txt_comanswer      證券
txt_top            證券
txt_answer         證券
txt_correct         T
title_comanswer    兩岸
title_answer       證券
title_correct       T
Name: 662, dtype: object
txt_comanswer      生活
txt_top            生活
txt_answer         地方
txt_correct         F
title_comanswer    兩岸
title_answer       地方
title_correct       T
Name: 663, dtype: object
txt_comanswer      證券
txt_top            證券
txt_answer         證券
txt_correct         T
title_comanswer  

txt_comanswer      證券
txt_top            證券
txt_answer         產經
txt_correct         T
title_comanswer    兩岸
title_answer       產經
title_correct       T
Name: 764, dtype: object
txt_comanswer      娛樂
txt_top            娛樂
txt_answer         生活
txt_correct         T
title_comanswer    生活
title_answer       生活
title_correct       T
Name: 765, dtype: object
txt_comanswer      政治
txt_top            政治
txt_answer         政治
txt_correct         T
title_comanswer    兩岸
title_answer       政治
title_correct       T
Name: 766, dtype: object
txt_comanswer      地方
txt_top            地方
txt_answer         地方
txt_correct         F
title_comanswer    兩岸
title_answer       地方
title_correct       T
Name: 767, dtype: object
txt_comanswer      運動
txt_top            運動
txt_answer         運動
txt_correct         T
title_comanswer    運動
title_answer       運動
title_correct       T
Name: 768, dtype: object
txt_comanswer      兩岸
txt_top            產經
txt_answer         產經
txt_correct         T
title_comanswer  

Name: 879, dtype: object
txt_comanswer      證券
txt_top            證券
txt_answer         證券
txt_correct         T
title_comanswer    證券
title_answer       證券
title_correct       T
Name: 880, dtype: object
txt_comanswer      生活
txt_top            生活
txt_answer         生活
txt_correct         F
title_comanswer    兩岸
title_answer       生活
title_correct       T
Name: 881, dtype: object
txt_comanswer      運動
txt_top            運動
txt_answer         運動
txt_correct         T
title_comanswer    運動
title_answer       運動
title_correct       T
Name: 882, dtype: object
txt_comanswer      娛樂
txt_top            娛樂
txt_answer         娛樂
txt_correct         T
title_comanswer    娛樂
title_answer       娛樂
title_correct       T
Name: 883, dtype: object
txt_comanswer      社會
txt_top            社會
txt_answer         社會
txt_correct         T
title_comanswer    兩岸
title_answer       社會
title_correct       T
Name: 884, dtype: object
txt_comanswer      文化
txt_top            文化
txt_answer         文化
txt_correct   

Name: 988, dtype: object
txt_comanswer      文化
txt_top            文化
txt_answer         文化
txt_correct         F
title_comanswer    文化
title_answer       文化
title_correct       T
Name: 989, dtype: object
txt_comanswer      運動
txt_top            運動
txt_answer         運動
txt_correct         T
title_comanswer    運動
title_answer       運動
title_correct       T
Name: 990, dtype: object
txt_comanswer      證券
txt_top            證券
txt_answer         證券
txt_correct         T
title_comanswer    證券
title_answer       證券
title_correct       T
Name: 991, dtype: object
txt_comanswer      生活
txt_top            生活
txt_answer         生活
txt_correct         F
title_comanswer    生活
title_answer       生活
title_correct       T
Name: 992, dtype: object
txt_comanswer      國際
txt_top            國際
txt_answer         國際
txt_correct         T
title_comanswer    兩岸
title_answer       國際
title_correct       T
Name: 993, dtype: object
txt_comanswer      產經
txt_top            產經
txt_answer         社會
txt_correct   

Name: 1108, dtype: object
txt_comanswer      科技
txt_top            科技
txt_answer         科技
txt_correct         T
title_comanswer    科技
title_answer       科技
title_correct       T
Name: 1109, dtype: object
txt_comanswer      文化
txt_top            文化
txt_answer         文化
txt_correct         F
title_comanswer    文化
title_answer       文化
title_correct       T
Name: 1110, dtype: object
txt_comanswer      地方
txt_top            地方
txt_answer         地方
txt_correct         F
title_comanswer    地方
title_answer       地方
title_correct       T
Name: 1111, dtype: object
txt_comanswer      娛樂
txt_top            娛樂
txt_answer         兩岸
txt_correct         T
title_comanswer    兩岸
title_answer       兩岸
title_correct       T
Name: 1112, dtype: object
txt_comanswer      文化
txt_top            文化
txt_answer         文化
txt_correct         F
title_comanswer    兩岸
title_answer       文化
title_correct       T
Name: 1113, dtype: object
txt_comanswer      社會
txt_top            社會
txt_answer         社會
txt_corr

In [41]:
dataset

[({'txt_top': '娛樂'}, '娛樂'),
 ({'txt_top': '兩岸'}, '兩岸'),
 ({'txt_top': '兩岸'}, '兩岸'),
 ({'txt_top': '社會'}, '社會'),
 ({'txt_top': '科技'}, '科技'),
 ({'txt_top': '運動'}, '運動'),
 ({'txt_top': '證券'}, '證券'),
 ({'txt_top': '娛樂'}, '娛樂'),
 ({'txt_top': '兩岸'}, '兩岸'),
 ({'txt_top': '娛樂'}, '娛樂'),
 ({'txt_top': '運動'}, '運動'),
 ({'txt_top': '文化'}, '文化'),
 ({'txt_top': '政治'}, '政治'),
 ({'txt_top': '生活'}, '生活'),
 ({'txt_top': '文化'}, '文化'),
 ({'txt_top': '兩岸'}, '兩岸'),
 ({'txt_top': '科技'}, '科技'),
 ({'txt_top': '政治'}, '政治'),
 ({'txt_top': '地方'}, '地方'),
 ({'txt_top': '地方'}, '地方'),
 ({'txt_top': '娛樂'}, '娛樂'),
 ({'txt_top': '兩岸'}, '兩岸'),
 ({'txt_top': '政治'}, '政治'),
 ({'txt_top': '科技'}, '科技'),
 ({'txt_top': '產經'}, '產經'),
 ({'txt_top': '產經'}, '產經'),
 ({'txt_top': '政治'}, '政治'),
 ({'txt_top': '證券'}, '證券'),
 ({'txt_top': '地方'}, '地方'),
 ({'txt_top': '社會'}, '社會'),
 ({'txt_top': '文化'}, '文化'),
 ({'txt_top': '生活'}, '生活'),
 ({'txt_top': '政治'}, '政治'),
 ({'txt_top': '證券'}, '證券'),
 ({'txt_top': '兩岸'}, '兩岸'),
 ({'txt_top': '證券'},

In [42]:
size = int(len(dataset)*0.7)
train_data, test_data = dataset[:size], dataset[size:]
test_data2 = [i[0] for i in test_data]

classif0 = SklearnClassifier(BernoulliNB()).train(train_data)
classif = SklearnClassifier(SVC(), sparse=False).train(train_data)

In [43]:
ch = []
n=0
for f in dataset[size:] :
    ch.append([f,classif.classify_many(test_data2)[n]])
    n += 1
ch

[[({'txt_top': '文化'}, '文化'), '文化'],
 [({'txt_top': '科技'}, '科技'), '科技'],
 [({'txt_top': '兩岸'}, '兩岸'), '兩岸'],
 [({'txt_top': '地方'}, '地方'), '地方'],
 [({'txt_top': '生活'}, '生活'), '生活'],
 [({'txt_top': '社會'}, '社會'), '社會'],
 [({'txt_top': '運動'}, '運動'), '運動'],
 [({'txt_top': '社會'}, '社會'), '社會'],
 [({'txt_top': '運動'}, '運動'), '運動'],
 [({'txt_top': '運動'}, '運動'), '運動'],
 [({'txt_top': '社會'}, '社會'), '社會'],
 [({'txt_top': '科技'}, '科技'), '科技'],
 [({'txt_top': '社會'}, '社會'), '社會'],
 [({'txt_top': '科技'}, '科技'), '科技'],
 [({'txt_top': '國際'}, '國際'), '國際'],
 [({'txt_top': '文化'}, '文化'), '文化'],
 [({'txt_top': '地方'}, '地方'), '地方'],
 [({'txt_top': '社會'}, '社會'), '社會'],
 [({'txt_top': '社會'}, '社會'), '社會'],
 [({'txt_top': '產經'}, '產經'), '產經'],
 [({'txt_top': '運動'}, '運動'), '運動'],
 [({'txt_top': '社會'}, '社會'), '社會'],
 [({'txt_top': '娛樂'}, '娛樂'), '娛樂'],
 [({'txt_top': '兩岸'}, '兩岸'), '兩岸'],
 [({'txt_top': '文化'}, '文化'), '文化'],
 [({'txt_top': '娛樂'}, '娛樂'), '娛樂'],
 [({'txt_top': '娛樂'}, '娛樂'), '娛樂'],
 [({'txt_top': '政治'}, '政治'),

### 將測試集帶入模型回傳預測類別，將預測類別與測試集正確類別比對，回傳正確率並列出錯誤文本

In [44]:
back = classif.classify_many(test_data2)
n = 0
p=0
while n < len(back):
    if back[n] ==  [i[1] for i in test_data][n]:
        p += 1
    
    else:
        print(back[n],' ',[i[1] for i in test_data][n],' ',[i[0] for i in test_data][n])
    n += 1
print(p)
print(p/len(back))

360
1.0


## 6. 定義get_lab()將兩次機器學習模型放進去，回傳預測結果

### get_lab( 標題 , 內文 ) = 預測類別


In [50]:
def get_lab(title='',txt='',top=200,get=30):
    f = pos_fwords(txt,False,top)
    t =  pos_fwords(title,True,top)
    txttop = list(get_top(txt,top,get).values())[0]

    if len(t.keys()) < 2:
        final = classif.classify_many({'txt_comanswer': txt_classifier.classify(f),'txt_top':txttop})
        final_title = "Na"
    elif title_classifier.classify(t) == txt_classifier.classify(f) and txt_classifier.classify(f)==top and top == title_classifier.classify(t):
        final = txt_classifier.classify(f)
        final_title = title_classifier.classify(t)
    else:
        final = classif.classify_many({'title_comanswer': title_classifier.classify(t), 'txt_comanswer': txt_classifier.classify(f),'txt_top':txttop})
        final_title = title_classifier.classify(t)
    print(f)
    print([final_title,txt_classifier.classify(f),txttop],'p')

    return final

In [51]:
def get_lab2(title='',txt='',top=200,get=30):
    f = pos_features(txt,False,top)
    t =  pos_features(title,True,top)
    txttop = list(get_top(txt,top,get).values())[0]

#    if len(t.keys()) < 2:
    final = classif.classify_many({'txt_comanswer': txt_classifier.classify(f),'txt_top':txttop})
#        final_title = "Na"
#    elif title_classifier.classify(t) == txt_classifier.classify(f) and txt_classifier.classify(f)==top and top == title_classifier.classify(t):
#        final = txt_classifier.classify(f)
#        final_title = title_classifier.classify(t)
#    else:
#        final = classif.classify_many({'title_comanswer': title_classifier.classify(t), 'txt_comanswer': txt_classifier.classify(f),'txt_top':txttop})
#        final_title = title_classifier.classify(t)
#    print(f)
#    print([final_title,txt_classifier.classify(f),txttop],'p')

    return final

## 7. 測試其他新聞網站的文章

In [58]:
test_title="""
昨探視江丙坤 柯Ｐ：學生叫他去看要不要撤掉管子


"""

In [59]:
test_txt="""
前海基會董事長江丙坤昨晚不幸病逝馬偕醫院，享壽86歲，過世前台北市長柯文哲有去探望他，媒體今（11）日問柯文哲，為何會去看江丙坤？柯文哲表示，馬偕醫院心臟外科醫師是他在台大的學生，遇到這種V.I.P.，他們比較不想自己做決定，所以叫老師去看，看到底要不要撤掉管子。



柯文哲說，自己曾經是重症醫學專家，因為比較年輕的醫師要去跟家屬說，這一定不會活，要撤管，當然有壓力，所以就由他去處理。媒體追問，「所以是你跟家屬說拔管嗎？」柯並未回應。



此外，台北市長驗票結果今天將出爐，國民黨候選人丁守中仍輸柯文哲3千多票，媒體今問柯，是否擔心丁守中提選舉無效訴訟，對此，柯文哲表示，他會去問律師，這要花多少錢，募的錢都快不夠了。



至於是否會私底下找丁守中，勸丁不要提選舉無效訴訟，柯則說，這不容易，這一次驗票 單單民政局就花了一千一百萬元，因為二備金他要簽，不曉得司法系統要花多少錢，然後競選總部的律師也要花錢，他猜丁守中花更多。
 


"""

In [60]:
get_lab(title=test_title,txt=test_txt,top=200)

[('政治', 2), ('科技', 2), ('地方', 2), ('文化', 2), ('生活', 2)]
txt_comanwer=兩岸
dict_values(['政治'])

{}
['Na', '兩岸', '政治'] p


['政治']

In [61]:
txt_classifier.classify(pos_fwords(test_txt))

'兩岸'

In [ ]:
stop

In [57]:
p = 0
for book in books:
    title = list(book[0].keys())[0]
    txt = list(book[1].keys())[0]
    A = get_lab(title=title,txt=txt,top=175,get=25)[0]
    print(list(book[0].values())[0],'/',A)
    print("")
    if list(book[0].values())[0] == A:
        p += 1
print(p/len(books))

[('娛樂', 11), ('地方', 1), ('產經', 1), ('國際', 1), ('文化', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

{'影視': True, '影視局': True, '影匯': True, '市場展': True, '優質': True, '造勢': True, '電視劇': True, '藝人': True, '人氣': True, '劇情': True, '主持': True}
['娛樂', '娛樂', '娛樂'] p
娛樂 / 娛樂

[('兩岸', 7), ('社會', 2), ('生活', 1), ('科技', 1), ('國際', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

{'宣誓': True, '電台': True, '依法': True, '自治區': True, '市': True, '發出': True, '維權': True}
['Na', '兩岸', '兩岸'] p
兩岸 / 兩岸

[('兩岸', 5), ('政治', 4), ('娛樂', 1), ('產經', 1), ('生活', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

{'邱垂正': True, '拜會': True, '上海市': True, '率團': True, '綜合': True}
['Na', '兩岸', '兩岸'] p
兩岸 / 兩岸

[('社會', 5), ('地方', 3), ('運動', 1), ('國際', 1)]
txt_comanwer=社會
dict_values(['社會'])

{'掩埋場': True, '偵辦': True, '北上': True, '北檢': True, '地檢署': True}
['社會', '社會', '社會'] p
社會 / 社會

[('科技', 3), ('娛樂', 3), ('生活', 3), ('國際', 3), ('產經', 2)]
txt_comanwer=生活
dict_values(['科技'])

{'有線': True, '管制': True, '男性': True}
['Na', '生活', '科技'] p
科技 / 科技

[('運動', 21)

[('科技', 10), ('生活', 3), ('證券', 3), ('文化', 2), ('娛樂', 2)]
txt_comanwer=科技
dict_values(['科技'])

{'近視': True, '科技部': True, '產學': True, 'AI': True, '深度': True, '用於': True, '揭開': True, '材料': True, '陳良基': True, '生物': True, '機械': True, '虛擬': True, '實境': True}
['科技', '科技', '科技'] p
科技 / 科技

[('國際', 4), ('兩岸', 2), ('生活', 1), ('社會', 1), ('運動', 1)]
txt_comanwer=國際
dict_values(['國際'])

{'大臣': True, '首相': True, '廣播': True, 'BBC': True}
['Na', '國際', '國際'] p
國際 / 國際

[('產經', 4), ('政治', 3), ('生活', 1), ('國際', 1), ('地方', 1)]
txt_comanwer=產經
dict_values(['產經'])

{'國發會': True, '主計': True, '總處': True, '費用': True}
['產經', '產經', '產經'] p
產經 / 產經

[('文化', 9), ('娛樂', 7), ('社會', 2), ('生活', 1), ('兩岸', 1)]
txt_comanwer=文化
dict_values(['文化'])

{'劇場': True, '劇團': True, '恐懼': True, '小說': True, '真實': True, '短篇': True, '倫敦': True, '首演': True, '感受': True}
['Na', '文化', '文化'] p
文化 / 文化

[('證券', 20), ('科技', 1), ('產經', 1)]
txt_comanwer=證券
dict_values(['證券'])

{'聯茂': True, '股價': True, '基板': True, '供應商': True, '欣興': True, '走揚':

[('科技', 11), ('產經', 4), ('生活', 1), ('國際', 1)]
txt_comanwer=科技
dict_values(['科技'])

{'衛星': True, '太空': True, '軌道': True, '地球': True, '太空船': True, '燃料': True, '發射': True, 'Space': True, '機器人': True, '數十': True, '通信': True}
['Na', '科技', '科技'] p
科技 / 科技

[('科技', 12), ('生活', 4), ('國際', 2), ('產經', 2), ('地方', 1)]
txt_comanwer=科技
dict_values(['科技'])

{'柯': True, '諾': True, '人類': True, '科學家': True, '地球': True, '生物': True, '航空': True, '太空': True, '總署': True, 'NASA': True, '旅行': True, '一步': True}
['科技', '科技', '科技'] p
科技 / 科技

[('國際', 5), ('兩岸', 2), ('社會', 2), ('證券', 1), ('產經', 1)]
txt_comanwer=國際
dict_values(['國際'])

{'盧廣': True, '監禁': True, '大使': True, '釋放': True, '允許': True}
['Na', '國際', '國際'] p
國際 / 國際

[('兩岸', 19), ('科技', 2), ('生活', 2), ('國際', 1), ('地方', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

{'寧夏': True, '民族': True, '穆斯林': True, '反恐': True, '回族': True, '自治區': True, '維穩': True, '手段': True, '打壓': True, '高層': True, '書記': True, '早': True, '處置': True, '網': True, '常委': True, '依法': True, '聲稱': Tr

[('娛樂', 6), ('政治', 5), ('社會', 3), ('運動', 3), ('生活', 3)]
txt_comanwer=娛樂
dict_values(['娛樂'])

{'安迪': True, '綜藝': True, '演藝': True, '藝人': True, '先生': True, '謝謝': True, '台視': True, '出道': True, '播出': True, '電視劇': True, '演戲': True, '主持': True, '專輯': True}
['Na', '娛樂', '娛樂'] p
娛樂 / 娛樂

[('證券', 17), ('科技', 3), ('兩岸', 2), ('產經', 2), ('文化', 1)]
txt_comanwer=證券
dict_values(['證券'])

{'多空': True, '雜陳': True, '美股': True, '景氣': True, '指數': True, '短期': True, '震盪': True, '整理': True, '台股': True, '外資': True, '概念股': True, '展望': True, '股市': True, '走勢': True, '休兵': True, '擔憂': True, '法人': True, '賣壓': True, '作帳': True, '波段': True, '接單': True, '車用': True}
['證券', '證券', '證券'] p
證券 / 證券

[('證券', 24), ('兩岸', 1)]
txt_comanwer=證券
dict_values(['證券'])

{'台股': True, '指數': True, '盤中': True, '外資': True, '股市': True, '震盪': True, '短線': True, '加權': True, '強勢': True, '分析師': True, '美股': True, '大跌': True, '翻紅': True, '蘋概股': True, '之上': True, '股價': True, '成交': True, '成交值': True, '台積電': True, '大立光': True, '鴻海': True, '科技股': Tru

[('社會', 16), ('娛樂', 1), ('生活', 1), ('運動', 1), ('科技', 1)]
txt_comanwer=社會
dict_values(['社會'])

{'計程車': True, '背包': True, '派出所': True, '警員': True, '警察': True, '循線': True, '警察局': True, '松山': True, '分局': True, '接獲': True, '不慎': True, '請求': True, '獲報': True, '乘客': True, '後座': True, '員警': True}
['Na', '社會', '社會'] p
社會 / 社會

[('文化', 13), ('娛樂', 3), ('證券', 2), ('地方', 2), ('科技', 2)]
txt_comanwer=文化
dict_values(['文化'])

{'紙風車': True, '劇團': True, '巫頂': True, '首演': True, '想像': True, '傳達': True, '片段': True, '時光': True, '感受': True, '李永豐': True, '長年': True, '藉著': True, '劇場': True}
['Na', '文化', '文化'] p
文化 / 文化

[('國際', 20), ('產經', 2), ('兩岸', 1), ('科技', 1), ('運動', 1)]
txt_comanwer=國際
dict_values(['國際'])

{'歐盟': True, '延後': True, '柯賓': True, '保守黨': True, '在野黨': True, '邊境': True, '首相': True, '愛爾蘭': True, '混亂': True, '英鎊': True, '領導人': True, 'Theresa': True, 'May': True, '延遲': True, '工黨': True, '黨魁': True, 'Jeremy': True, '下議院': True, '北愛爾蘭': True, '否決': True, '布魯塞爾': True, '遭遇': True, '一再': True, 'John':

...

[('文化', 14), ('兩岸', 1), ('地方', 1), ('政治', 1), ('證券', 1)]
txt_comanwer=文化
dict_values(['文化'])

{'服裝': True, '時裝週': True, '戲曲': True, '模特兒': True, '走秀': True, '傳達': True, '起至': True, '細緻': True, '文化局': True, '舞者': True, '劇場': True, '感受': True, '海': True, '搭配': True}
['Na', '文化', '文化'] p
文化 / 文化

[('政治', 7), ('社會', 4), ('科技', 1), ('地方', 1)]
txt_comanwer=政治
dict_values(['政治'])

{'馬偕': True, '江丙坤': True, '加護': True, '病房': True, '意願': True, '海峽': True, '昏迷': True}
['兩岸', '政治', '政治'] p
政治 / 政治

[('證券', 11), ('產經', 9), ('生活', 2), ('文化', 2), ('政治', 1)]
txt_comanwer=證券
dict_values(['證券'])

{'KY': True, '合併': True, '力道': True, '加速': True, '出貨': True, '挹注': True, '產能': True, '暢旺': True, '展望': True, '代工': True, '接單': True}
['Na', '證券', '證券'] p
產經 / 證券

[('娛樂', 16), ('文化', 2), ('政治', 2), ('兩岸', 1), ('科技', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

{'喜劇': True, '好萊塢': True, '喜劇類': True, '金球獎': True, '飾演': True, '片': True, '男星': True, '配角': True, '愛情': True, '巨星': True, '黑豹': True, '娛樂': True, 'Awards':

[('國際', 9), ('兩岸', 2), ('社會', 2), ('娛樂', 1), ('科技', 1)]
txt_comanwer=國際
dict_values(['國際'])

{'皮查': True, '伊': True, '眾議院': True, '抨擊': True, '稍早': True, '具體': True, '細節': True, '打算': True, '眾多': True}
['Na', '國際', '國際'] p
國際 / 國際

[('文化', 2), ('兩岸', 2), ('地方', 2), ('運動', 2), ('政治', 1)]
txt_comanwer=兩岸
dict_values(['兩岸'])

{}
['Na', '兩岸', '兩岸'] p
政治 / 兩岸

[('娛樂', 10), ('文化', 3), ('運動', 2), ('地方', 1), ('國際', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

{'電影節': True, '劇情': True, '放映': True, '金馬獎': True, '演技': True, '戲院': True, '超': True, '演戲': True, '飾演': True, '配角': True}
['Na', '娛樂', '娛樂'] p
娛樂 / 娛樂

[('國際', 10), ('產經', 3), ('證券', 2), ('科技', 2)]
txt_comanwer=國際
dict_values(['國際'])

{'伊朗': True, '引渡': True, '知情': True, '制裁': True, '彭博': True, '威脅': True, '尋求': True, '單一': True, '兩國': True, '眾議院': True}
['Na', '國際', '國際'] p
產經 / 國際

[('產經', 4), ('兩岸', 2), ('科技', 1), ('娛樂', 1), ('地方', 1)]
txt_comanwer=產經
dict_values(['產經'])

{'陸製': True, '訂定': True, '危機': True, '許可': True}
['Na', '產經', '產經'] 

[('生活', 4), ('科技', 1), ('地方', 1), ('社會', 1), ('國際', 1)]
txt_comanwer=生活
dict_values(['生活'])

{'CBS': True, '玩': True, '衛生': True, '家長': True}
['Na', '生活', '生活'] p
生活 / 生活

[('文化', 12), ('科技', 3), ('政治', 2), ('運動', 1), ('兩岸', 1)]
txt_comanwer=文化
dict_values(['文化'])

{'畫家': True, '美術': True, '油畫': True, '寫生': True, '書法': True, '文化局': True, '理事長': True, '畫作': True, '藝文': True, '美麗': True, '保存': True, '建築': True}
['Na', '文化', '文化'] p
文化 / 文化

[('文化', 14), ('娛樂', 4), ('地方', 2), ('國際', 2), ('運動', 1)]
txt_comanwer=文化
dict_values(['文化'])

{'歌仔戲': True, '周洪': True, '明雪': True, '藝師': True, '歷經': True, '戲曲': True, '文化局': True, '一生': True, '傳承': True, '登錄': True, '出生': True, '金枝': True, '保存': True, '劇團': True}
['兩岸', '文化', '文化'] p
文化 / 文化

[('科技', 21), ('國際', 4)]
txt_comanwer=科技
dict_values(['科技'])

{'太空': True, '太空人': True, '火箭': True, '太空站': True, '總署': True, '發射': True, '聯合號': True, '失敗': True, '載人': True, '飛行': True, '任務': True, '諾': True, '搭載': True, '柯': True, '航空': True, 'NASA': True, 'Spac

[('生活', 10), ('地方', 2), ('政治', 1), ('國際', 1)]
txt_comanwer=生活
dict_values(['生活'])

{'勞動部': True, '許銘春': True, '諮詢': True, '假期': True, '客運': True, '翻轉': True, '職涯': True, '運輸': True, '疏運': True, '補充': True}
['Na', '生活', '生活'] p
生活 / 生活

[('運動', 24), ('地方', 1)]
txt_comanwer=運動
dict_values(['運動'])

{'湖人': True, '馬刺': True, '領先': True, '詹姆斯': True, '分球': True, '三分球': True, '擊敗': True, '上籃': True, '差距': True, '得分': True, '投': True, '職籃': True, 'NBA': True, '主場': True, '時刻': True, '半場': True, '最多': True, '打完': True, '助': True, '攻勢': True, '落後': True, '超前': True, '進攻': True, '比數': True, '擊退': True, '上場': True, '助攻': True, '籃板': True, '全場': True, '球季': True, '交手': True, '球員': True, '本季': True, '爵士': True, '吞下': True, '敗': True, '前鋒': True, '首節': True, '退場': True}
['運動', '運動', '運動'] p
運動 / 運動

[('地方', 4), ('社會', 1), ('兩岸', 1), ('科技', 1), ('國際', 1)]
txt_comanwer=地方
dict_values(['地方'])

{'議長': True, '基隆市': True, '席次': True, '副議長': True}
['Na', '地方', '地方'] p
地方 / 地方

[('科技', 8), ('證券', 2), ('產經', 

[('運動', 6), ('娛樂', 3), ('生活', 2), ('文化', 2), ('地方', 2)]
txt_comanwer=運動
dict_values(['運動'])

{'身材': True, '球團': True, '職棒': True, 'Lamigo': True, '桃猿隊': True, '平時': True}
['Na', '運動', '運動'] p
運動 / 運動

[('地方', 11), ('社會', 1), ('科技', 1), ('兩岸', 1), ('產經', 1)]
txt_comanwer=地方
dict_values(['地方'])

{'殯葬': True, '謠言': True, '專區': True, '鰲峰山': True, '分區': True, '民政局': True, '興建': True, '設施': True, '興辦': True, '都市': True, '市民': True}
['地方', '地方', '地方'] p
地方 / 地方

[('娛樂', 13), ('科技', 3), ('地方', 3), ('產經', 2), ('文化', 2)]
txt_comanwer=娛樂
dict_values(['娛樂'])

{'華視': True, '莊豐': True, '嘉': True, '綜藝': True, '金鐘': True, '風波': True, '一年': True, '邀': True, '藝人': True, '主持': True, '收視': True, '歌手': True, '製作人': True}
['兩岸', '娛樂', '娛樂'] p
娛樂 / 娛樂

[('娛樂', 16), ('證券', 4), ('生活', 1), ('科技', 1), ('地方', 1)]
txt_comanwer=娛樂
dict_values(['娛樂'])

{'悲傷': True, '票房': True, '破億': True, '戲院': True, '上映': True, '編劇': True, '主演': True, '繼': True, '花甲': True, '超': True, '娛樂': True, 'KKBOX': True, '單曲': True, '愛情': Tr